In [1]:
pip install pycatch22


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sktime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 22.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 22.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Imports

In [4]:
from collections import Counter
from sklearn.metrics import accuracy_score, matthews_corrcoef, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn import linear_model
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from scipy import stats
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
import math
from math import sqrt
import statistics 
import csv
from sklearn.dummy import DummyClassifier
from pycatch22 import catch22_all
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.covariance import EllipticEnvelope
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.cluster import AgglomerativeClustering
from scipy.stats import ks_2samp
from scipy.stats import ranksums 
from scipy.stats import wasserstein_distance
from scipy.special import rel_entr
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE



%matplotlib inline

filename0 = '/work/data/Dataset_0_new.csv'
data0 = pd.read_csv(filename0)
print("Number of lines present:", len(data0))

filename1 = '/work/data/Dataset_1_new.csv'
data1 = pd.read_csv(filename1)

filename2 = '/work/data/Dataset_2_new.csv'
data2 = pd.read_csv(filename2)

filename3 = '/work/data/Dataset_3_new.csv'
data3 = pd.read_csv(filename3)

filename4 = '/work/data/Dataset_4_new.csv'
data4 = pd.read_csv(filename4)

all_data = [data0, data1, data2, data3, data4]

real_change_points = all_data[0]['Fraud_point'].values
consumer_ids = all_data[0]['Customer_ID'].values

print(consumer_ids)


2023-11-07 04:51:59.433544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 04:51:59.632876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-07 04:51:59.632933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-07 04:51:59.682865: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 04:52:01.822129: W tensorflow/stream_executor/platform/de

# Functions

In [5]:
# Get consumer data, condumer ID and fraud point
def consumerData(data, index):
    consumer_data = data.iloc[index,3:].values
    consumer_id = data['Customer_ID'][index]
    fraud_point = data['Fraud_point'][index]
#     print(consumer_data)
    return consumer_data, consumer_id, fraud_point
        
def setupWeeklyData(consumer_data):
    day = 96
    week_period = 7 # number of consecutive days to detect a change
    week = 7*96
    year = 96 * 365
    window_len = day*week_period # window length for the statistical tests

    # Data weekly from first year
    weekly_data1year = [] # (X_train)
    # Data weekly from second year
    weekly_data2year = [] # (X_test)

    for i in range(0, year-(7*96), day): 
        weekly_data1year.append(consumer_data[i:i+window_len])
        weekly_data2year.append(consumer_data[i+year:(i+year)+window_len])
        
    # Convert data into a list
#     weekly_data1year = np.array(weekly_data1year)
#     weekly_data2year = np.array(weekly_data2year)
    return weekly_data1year, weekly_data2year

def get_ground_truth(num_time_points, fraud_point_week):
    ground_truth = [0] * num_time_points
    if 0 <= fraud_point_week < num_time_points:
        ground_truth[int(fraud_point_week)] = 1
    ground_truth[int(fraud_point_week):] = [1] * len(ground_truth[int(fraud_point_week):])
    return ground_truth

def measure_accuracy(predicted_change_weeks, fraud_point_week):
    TP = 0
    FP = 0
    
    print(predicted_change_weeks, fraud_point_week)
    for week in predicted_change_weeks:
        if abs(week - fraud_point_week) <= 1: # assuming a tolerance of 1 week
            TP += 1
        else:
            FP += 1
    
    FN = 0 if TP > 0 else 1  # If we didn't find the fraud point, then it's a False Negative
    TN = len(predicted_change_weeks) - (TP + FP)  # All other weeks are True Negatives
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    if TP + FP != 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    recall = TP / (TP + FN)
    if precision + recall != 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0

    return accuracy, precision, recall, f1

# Stastical

## Wasserstein Distance

In [6]:
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
day = 96
week_period = 1

true_labels = []
predicted_labels = []

print('Wasserstein Distance\n\n')

count = 0
for data in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(data.index)):
        consumer_data, consumer_id, fraud_point = consumerData(data, j)
        fraud_point_day = int((fraud_point/96) - 365)
        year_counter = 0
        year_results = []
        window_len = day*week_period # Window length for the statistical tests
        metrics_list = []
        threshold = 0.1
        fraud_point_week = (fraud_point/96)
        results = [] # Store the results of the statistical test
        count = 0 # Counter to verify if the distributions are different during N consecutive days
        w0 = consumer_data[0:window_len] # set the data of the initial window w0
        for i in range(0, len(consumer_data), day):
            w1 = consumer_data[i:i+window_len]
            distance = wasserstein_distance(w0, w1)
            results.append(distance)
                
            # year_counter = year_counter + 1
            # if year_counter == 365:
            #     yearly_mean = statistics.mean(results)
            #     yearly_sd = statistics.stdev(results)
            #     new_threshold = yearly_mean * (2*yearly_sd)
            #     threshold = new_threshold
            #     month_counter = 0
        
        true_labels = [1 if i >= fraud_point_day else 0 for i in range(len(results))]
        predicted_labels = [1 if w_distance > threshold else 0 for w_distance in results]

        # print(true_labels, '\n\n')
        # print(predicted_labels, '\n\n')

        accuracy = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels)
        recall = recall_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Wasserstein Distance


Dataset: 1
Consumer: 27396
Accuracy: 0.87
Precision: 0.89
Recall: 0.98
F1 Score: 0.93
--------------
Consumer: 31709
Accuracy: 0.86
Precision: 0.92
Recall: 0.92
F1 Score: 0.92
--------------
Consumer: 19373
Accuracy: 0.86
Precision: 0.86
Recall: 1.00
F1 Score: 0.93
--------------
Consumer: 4691
Accuracy: 0.97
Precision: 0.99
Recall: 0.98
F1 Score: 0.98
--------------
Consumer: 11612
Accuracy: 0.93
Precision: 0.96
Recall: 0.97
F1 Score: 0.96
--------------
Consumer: 27966
Accuracy: 0.78
Precision: 0.87
Recall: 0.88
F1 Score: 0.88
--------------
Consumer: 28310
Accuracy: 0.94
Precision: 0.95
Recall: 0.99
F1 Score: 0.97
--------------
Consumer: 27391
Accuracy: 0.96
Precision: 0.98
Recall: 0.98
F1 Score: 0.98
--------------
Consumer: 30199
Accuracy: 0.87
Precision: 0.89
Recall: 0.96
F1 Score: 0.93
--------------
Consumer: 26952
Accuracy: 0.78
Precision: 0.80
Recall: 0.97
F1 Score: 0.88
--------------
Consumer: 30894
Accuracy: 0.97
Precision: 0.98
Recall: 0.99
F1 Scor

# Catch 22 - ML

## Isolation Forest

In [ ]:

# Create a StandardScaler instance
scaler = StandardScaler()

# Deciding to keep the first N principal components
N_COMPONENTS = 22
pca = PCA(n_components=N_COMPONENTS)

total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
isolation_pred = []

print('Isolation Forest\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2: # Summing up to use features
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)

        smote = SMOTE(random_state=42)

        # Aplicar o over-sampling ao conjunto de treinamento
        features = smote.fit_resample(features)
        
        isf = IsolationForest(max_samples=80, random_state=2, n_estimators=50, contamination=float(0.1))
        isf.fit(features)
        pred = isf.predict(features)

        # Convert predictions to match ground truth format: 1 for anomalies and 0 for normal
        pred[pred == 1] = 0
        pred[pred == -1] = 1

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        # Calculate accuracy measures
        pred = pred.tolist()
        isolation_pred = pred
        # print("Pred: ", pred, "ground truth: ", ground_truth)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]


    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Isolation Forest


Dataset: 1
Consumer: 27396
Accuracy: 0.63
Precision: 0.51
Recall: 0.14
F1 Score: 0.22
--------------
Consumer: 31709
Accuracy: 0.60
Precision: 0.36
Recall: 0.10
F1 Score: 0.15
--------------
Consumer: 19373
Accuracy: 0.64
Precision: 0.42
Recall: 0.12
F1 Score: 0.19
--------------
Consumer: 4691
Accuracy: 0.59
Precision: 0.86
Recall: 0.18
F1 Score: 0.30
--------------
Consumer: 11612
Accuracy: 0.53
Precision: 0.36
Recall: 0.08
F1 Score: 0.13
--------------
Consumer: 27966
Accuracy: 0.67
Precision: 0.57
Recall: 0.16
F1 Score: 0.26
--------------
Consumer: 28310
Accuracy: 0.59
Precision: 0.42
Recall: 0.11
F1 Score: 0.17
--------------
Consumer: 27391
Accuracy: 0.47
Precision: 0.15
Recall: 0.03
F1 Score: 0.05
--------------
Consumer: 30199
Accuracy: 0.58
Precision: 0.22
Recall: 0.06
F1 Score: 0.10
--------------
Consumer: 26952
Accuracy: 0.62
Precision: 0.03
Recall: 0.01
F1 Score: 0.01
--------------
Consumer: 30894
Accuracy: 0.57
Precision: 0.68
Recall: 0.15
F1 Score: 0

In [ ]:
IsolationForest

## One Class SVM 

In [ ]:
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
oneclass_pred = []

print('One Class SVM\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)

         # Initialize OneClassSVM
        clf = OneClassSVM(kernel="rbf", nu=0.3)
        clf.fit(features)

        # Predict the anomalies
        pred = clf.predict(features)

        # Convert predictions to match ground truth format: 1 for anomalies and 0 for normal
        pred[pred == 1] = 0
        pred[pred == -1] = 1

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        # Calculate accuracy measures
        pred = pred.tolist()
        oneclass_pred = pred
        # print("Pred: ", pred, "ground truth: ", ground_truth)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]


    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

One Class SVM


Dataset: 1
Consumer: 27396
Accuracy: 0.62
Precision: 0.50
Recall: 0.40
F1 Score: 0.44
--------------
Consumer: 31709
Accuracy: 0.58
Precision: 0.42
Recall: 0.33
F1 Score: 0.37
--------------
Consumer: 19373
Accuracy: 0.55
Precision: 0.34
Recall: 0.29
F1 Score: 0.31
--------------
Consumer: 4691
Accuracy: 0.65
Precision: 0.71
Recall: 0.45
F1 Score: 0.55
--------------
Consumer: 11612
Accuracy: 0.48
Precision: 0.37
Recall: 0.25
F1 Score: 0.29
--------------
Consumer: 27966
Accuracy: 0.68
Precision: 0.54
Recall: 0.46
F1 Score: 0.50
--------------
Consumer: 28310
Accuracy: 0.59
Precision: 0.46
Recall: 0.35
F1 Score: 0.40
--------------
Consumer: 27391
Accuracy: 0.44
Precision: 0.33
Recall: 0.21
F1 Score: 0.26
--------------
Consumer: 30199
Accuracy: 0.41
Precision: 0.13
Recall: 0.11
F1 Score: 0.12
--------------
Consumer: 26952
Accuracy: 0.54
Precision: 0.21
Recall: 0.22
F1 Score: 0.22
--------------
Consumer: 30894
Accuracy: 0.55
Precision: 0.52
Recall: 0.34
F1 Score: 0.41

## SGD One Class SVM

In [ ]:
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
sgd_pred = []

print('SGD One Class SVM\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)
        
        sdgone = linear_model.SGDOneClassSVM(random_state=3, learning_rate='optimal')
        sdgone.fit(features)

        # Predict the anomalies
        pred = sdgone.predict(features)

        # Convert predictions to match ground truth format: 1 for anomalies and 0 for normal
        pred[pred == 1] = 0
        pred[pred == -1] = 1

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        # Calculate accuracy measures
        pred = pred.tolist()
        sgd_pred = pred
        # print("Pred: ", pred, "ground truth: ", ground_truth)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]


    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

SGD One Class SVM


Dataset: 1
Consumer: 27396
Accuracy: 0.18
Precision: 0.15
Recall: 0.25
F1 Score: 0.18
--------------
Consumer: 31709
Accuracy: 0.44
Precision: 0.33
Recall: 0.50
F1 Score: 0.40
--------------
Consumer: 19373
Accuracy: 0.63
Precision: 0.48
Recall: 0.72
F1 Score: 0.57
--------------
Consumer: 4691
Accuracy: 0.36
Precision: 0.35
Recall: 0.40
F1 Score: 0.37
--------------
Consumer: 11612
Accuracy: 0.32
Precision: 0.31
Recall: 0.45
F1 Score: 0.37
--------------
Consumer: 27966
Accuracy: 0.22
Precision: 0.13
Recall: 0.23
F1 Score: 0.17
--------------
Consumer: 28310
Accuracy: 0.34
Precision: 0.24
Recall: 0.31
F1 Score: 0.27
--------------
Consumer: 27391
Accuracy: 0.41
Precision: 0.36
Recall: 0.36
F1 Score: 0.36
--------------
Consumer: 30199
Accuracy: 0.67
Precision: 0.53
Recall: 0.84
F1 Score: 0.65
--------------
Consumer: 26952
Accuracy: 0.37
Precision: 0.15
Recall: 0.26
F1 Score: 0.19
--------------
Consumer: 30894
Accuracy: 0.46
Precision: 0.42
Recall: 0.41
F1 Score: 

## Local Outlier Factor

In [ ]:
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
lof_pred = []

print('Local Outlier Factor\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)
        
        lof = LocalOutlierFactor(n_neighbors=50, algorithm='auto', contamination=float(0.1), novelty=True)
        lof.fit(features)
        pred = lof.predict(features)

        # Convert predictions to match ground truth format: 1 for anomalies and 0 for normal
        pred[pred == 1] = 0
        pred[pred == -1] = 1

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        # Calculate accuracy measures
        pred = pred.tolist()
        lof_pred = pred
        # print("Pred: ", pred, "ground truth: ", ground_truth)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]


    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Local Outlier Factor


Dataset: 1
Consumer: 27396
Accuracy: 0.59
Precision: 0.28
Recall: 0.07
F1 Score: 0.11
--------------
Consumer: 31709
Accuracy: 0.60
Precision: 0.36
Recall: 0.09
F1 Score: 0.15
--------------
Consumer: 19373
Accuracy: 0.61
Precision: 0.30
Recall: 0.08
F1 Score: 0.13
--------------
Consumer: 4691
Accuracy: 0.51
Precision: 0.46
Recall: 0.09
F1 Score: 0.16
--------------
Consumer: 11612
Accuracy: 0.56
Precision: 0.52
Recall: 0.12
F1 Score: 0.19
--------------
Consumer: 27966
Accuracy: 0.61
Precision: 0.28
Recall: 0.08
F1 Score: 0.12
--------------
Consumer: 28310
Accuracy: 0.59
Precision: 0.39
Recall: 0.09
F1 Score: 0.15
--------------
Consumer: 27391
Accuracy: 0.50
Precision: 0.29
Recall: 0.06
F1 Score: 0.10
--------------
Consumer: 30199
Accuracy: 0.62
Precision: 0.38
Recall: 0.10
F1 Score: 0.15
--------------
Consumer: 26952
Accuracy: 0.63
Precision: 0.06
Recall: 0.02
F1 Score: 0.03
--------------
Consumer: 30894
Accuracy: 0.50
Precision: 0.34
Recall: 0.07
F1 Scor

## KMeans

In [ ]:
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
kmeans_pred = []

print('KMeans\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)
        features = features[~np.isnan(features).any(axis=1)]
        features = scaler.fit_transform(features)
        features = pca.fit_transform(features)

        # Using KMeans clustering
        kmeans = KMeans(n_clusters=2, n_init=10)  # using 2 clusters; normal and anomaly
        kmeans.fit(features)
        distances = np.linalg.norm(features - kmeans.cluster_centers_[kmeans.labels_], axis=1)
        anomaly_threshold = np.percentile(distances, 90)
        pred = np.where(distances > anomaly_threshold, 1, 0)  # 1 for anomalies and 0 for normal
        kmeans_pred = pred

        ground_truth = get_ground_truth(len(features), fraud_point_week)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        # Assuming total_accuracy, total_precision, total_recall, and total_f1 are lists that have been defined prior to the loop
        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}')
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}')
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}')
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}')

KMeans


Dataset: 1
Consumer: 27396
Accuracy: 0.63
Precision: 0.51
Recall: 0.14
F1 Score: 0.22
--------------
Consumer: 31709
Accuracy: 0.57
Precision: 0.24
Recall: 0.06
F1 Score: 0.10
--------------
Consumer: 19373
Accuracy: 0.57
Precision: 0.10
Recall: 0.03
F1 Score: 0.04
--------------
Consumer: 4691
Accuracy: 0.55
Precision: 0.65
Recall: 0.14
F1 Score: 0.23
--------------
Consumer: 11612
Accuracy: 0.50
Precision: 0.21
Recall: 0.05
F1 Score: 0.08
--------------
Consumer: 27966
Accuracy: 0.70
Precision: 0.74
Recall: 0.21
F1 Score: 0.33
--------------
Consumer: 28310
Accuracy: 0.57
Precision: 0.31
Recall: 0.08
F1 Score: 0.13
--------------
Consumer: 27391
Accuracy: 0.47
Precision: 0.18
Recall: 0.04
F1 Score: 0.06
--------------
Consumer: 30199
Accuracy: 0.58
Precision: 0.24
Recall: 0.07
F1 Score: 0.10
--------------
Consumer: 26952
Accuracy: 0.65
Precision: 0.18
Recall: 0.06
F1 Score: 0.09
--------------
Consumer: 30894
Accuracy: 0.55
Precision: 0.56
Recall: 0.12
F1 Score: 0.20
------

## Elliptic Envelope

In [ ]:
count = 0
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
elliptic_pred = []

print('Elliptic Envelope\n\n')

for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)
        
         # Using EllipticEnvelope (Minimum Covariance Determinant) for anomaly detection
        envelope = EllipticEnvelope(contamination=0.1, support_fraction=0.8)
        pred = envelope.fit_predict(features)
        
        # Convert predictions to match ground truth format: 1 for anomalies and 0 for normal
        pred[pred == 1] = 0  # inliers (normal points)
        pred[pred == -1] = 1  # outliers (anomalies)
        elliptic_pred = pred
        
        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Elliptic Envelope


Dataset: 1
Consumer: 27396
Accuracy: 0.66
Precision: 0.68
Recall: 0.18
F1 Score: 0.29
--------------
Consumer: 31709
Accuracy: 0.54
Precision: 0.06
Recall: 0.01
F1 Score: 0.02
--------------
Consumer: 19373
Accuracy: 0.68
Precision: 0.62
Recall: 0.18
F1 Score: 0.28
--------------
Consumer: 4691
Accuracy: 0.61
Precision: 0.94
Recall: 0.20
F1 Score: 0.33
--------------
Consumer: 11612
Accuracy: 0.48
Precision: 0.12
Recall: 0.03
F1 Score: 0.05
--------------
Consumer: 27966
Accuracy: 0.61
Precision: 0.31
Recall: 0.09
F1 Score: 0.14
--------------
Consumer: 28310
Accuracy: 0.57
Precision: 0.28
Recall: 0.07
F1 Score: 0.11
--------------
Consumer: 27391
Accuracy: 0.46
Precision: 0.11
Recall: 0.02
F1 Score: 0.04
--------------
Consumer: 30199
Accuracy: 0.59
Precision: 0.26
Recall: 0.07
F1 Score: 0.11
--------------
Consumer: 26952
Accuracy: 0.64
Precision: 0.12
Recall: 0.04
F1 Score: 0.06
--------------
Consumer: 30894
Accuracy: 0.53
Precision: 0.46
Recall: 0.10
F1 Score: 

## Autoencoders

In [ ]:
count = 0
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
autoencoder_pred = []

print('Autoencoders\n\n')

for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)
        
        input_dim = features.shape[1]
        encoding_dim = int(input_dim / 2)


        input_layer = Input(shape=(input_dim,))
        encoder = Dense(encoding_dim, activation="tanh")(input_layer)
        decoder = Dense(input_dim, activation="sigmoid")(encoder)

        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error")
        
        # Train the autoencoder
        autoencoder.fit(features, features, epochs=50, batch_size=20, validation_split=0.1, verbose=0)

        # Use the autoencoder to reconstruct the input
        predictions = autoencoder.predict(features)

        # Calculate reconstruction errors
        mse = np.mean(np.power(features - predictions, 2), axis=1)
        threshold = np.quantile(mse, 0.95)  # Here, we're setting a threshold at the 95th percentile of the error.
        pred = [1 if error > threshold else 0 for error in mse]
        autoencoder_pred = pred

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Autoencoders


Dataset: 1
2023-11-06 06:05:06.714955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-06 06:05:06.714990: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-06 06:05:06.715011: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-d7f56da1-e665-45c2-b1a9-a2c9d93d2a27): /proc/driver/nvidia/version does not exist
2023-11-06 06:05:06.715323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
23/23 [==============================]

## Agglomerative Clustering

In [ ]:
count = 0
total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
aggclust_pred = []

print('Agglomerative Clustering\n\n')

for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        features = []
        for week_data in weekly_consumption_year1 + weekly_consumption_year2:
            result = catch22_all(week_data)
            features.append(result['values'])

        features = np.array(features)

        # Check and handle NaN values 
        features = features[~np.isnan(features).any(axis=1)]

        # Normalize the features
        features = scaler.fit_transform(features)

        # Applying PCA transformation
        features = pca.fit_transform(features)
        
        # Using Agglomerative Clustering
        clustering = AgglomerativeClustering(n_clusters=2).fit(features)

        # Assuming the last cluster is anomalous
        pred = [1 if label == np.max(clustering.labels_) else 0 for label in clustering.labels_]
        aggclust_pred = pred

        # Get the ground truth
        ground_truth = get_ground_truth(len(features), fraud_point_week)

        accuracy = accuracy_score(ground_truth, pred)
        precision = precision_score(ground_truth, pred)
        recall = recall_score(ground_truth, pred)
        f1 = f1_score(ground_truth, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

        num_time_points = len(features)
        num_features = features.shape[1]

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}', )
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}', )
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}', )
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}', )

Agglomerative Clustering


Dataset: 1
Consumer: 27396
Accuracy: 0.01
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
--------------
Consumer: 31709
Accuracy: 0.98
Precision: 0.97
Recall: 0.98
F1 Score: 0.98
--------------
Consumer: 19373
Accuracy: 0.99
Precision: 0.97
Recall: 1.00
F1 Score: 0.98
--------------
Consumer: 4691
Accuracy: 0.75
Precision: 0.89
Recall: 0.55
F1 Score: 0.68
--------------
Consumer: 11612
Accuracy: 0.28
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
--------------
Consumer: 27966
Accuracy: 0.02
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
--------------
Consumer: 28310
Accuracy: 0.90
Precision: 0.80
Recall: 0.98
F1 Score: 0.88
--------------
Consumer: 27391
Accuracy: 0.48
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
--------------
Consumer: 30199
Accuracy: 0.57
Precision: 0.41
Recall: 0.39
F1 Score: 0.40
--------------
Consumer: 26952
Accuracy: 0.49
Precision: 0.07
Recall: 0.06
F1 Score: 0.07
--------------
Consumer: 30894
Accuracy: 0.80
Precision: 0.90
Recall: 0.64
F1 

## Isolation Forest + Local Outlier Factor

In [ ]:
# Combined models

combined_pred = np.logical_or(isolation_pred, lof_pred).astype(int)

accuracy = accuracy_score(ground_truth, combined_pred)
precision = precision_score(ground_truth, combined_pred)
recall = recall_score(ground_truth, combined_pred)
f1 = f1_score(ground_truth, combined_pred)

print(f"Consumer: {consumer_id}")
print(f"Combined Accuracy: {accuracy:.2f}")
print(f"Combined Precision: {precision:.2f}")
print(f"Combined Recall: {recall:.2f}")
print(f"Combined F1 Score: {f1:.2f}")
print("--------------")

total_accuracy.append(accuracy)
total_precision.append(precision)
total_recall.append(recall)
total_f1.append(f1)

Consumer: 45806
Combined Accuracy: 0.58
Combined Precision: 0.58
Combined Recall: 0.19
Combined F1 Score: 0.29
--------------


## Isolation Forest + Kmeans

In [ ]:
# Combined models

combined_pred = np.logical_or(isolation_pred, kmeans_pred).astype(int)

accuracy = accuracy_score(ground_truth, combined_pred)
precision = precision_score(ground_truth, combined_pred)
recall = recall_score(ground_truth, combined_pred)
f1 = f1_score(ground_truth, combined_pred)

print(f"Consumer: {consumer_id}")
print(f"Combined Accuracy: {accuracy:.2f}")
print(f"Combined Precision: {precision:.2f}")
print(f"Combined Recall: {recall:.2f}")
print(f"Combined F1 Score: {f1:.2f}")
print("--------------")

total_accuracy.append(accuracy)
total_precision.append(precision)
total_recall.append(recall)
total_f1.append(f1)

Consumer: 45806
Combined Accuracy: 0.57
Combined Precision: 0.54
Combined Recall: 0.17
Combined F1 Score: 0.25
--------------


## Wasserstein + Isolation Forest

In [ ]:
# Assuming `results` contains Wasserstein distances for each window
features_wasserstein = np.array(results).reshape(-1, 1)  # Reshape to use as a feature

# Combine with other features if needed
# features = np.hstack((features_wasserstein, other_features))

# Normalize the features including Wasserstein distances
features = scaler.fit_transform(features_wasserstein)

# Apply PCA only if there are more than one feature
if features.shape[1] > 1:
    pca = PCA(n_components=min(features.shape[0], features.shape[1]))
    features = pca.fit_transform(features)

# Train the Isolation Forest on the combined feature set
isf = IsolationForest(max_samples='auto', random_state=2, n_estimators=50, contamination=float(0.1))
isf.fit(features)

# Make predictions
pred = isf.predict(features)

# Convert predictions to match ground truth format
pred = np.where(pred == 1, 0, 1)  # Switching -1 to 1 for anomalies and 1 to 0 for normal

if len(pred) > len(ground_truth):
    pred = pred[:len(ground_truth)]

accuracy = accuracy_score(ground_truth, pred)
precision = precision_score(ground_truth, pred)
recall = recall_score(ground_truth, pred)
f1 = f1_score(ground_truth, pred)

print(f"Consumer: {consumer_id}")
print(f"Combined Accuracy: {accuracy:.2f}")
print(f"Combined Precision: {precision:.2f}")
print(f"Combined Recall: {recall:.2f}")
print(f"Combined F1 Score: {f1:.2f}")
print("--------------")



Consumer: 45806
Combined Accuracy: 0.50
Combined Precision: 0.23
Combined Recall: 0.05
Combined F1 Score: 0.09
--------------


In [ ]:
def setupXData(consumer_data):
    day = 96
    week_period = 7 # number of consecutive days to detect a change
    ncp = 20 # ncp -> 100 consumers, 20 changes (ncp = 20)
    year = 96 * 365
    window_len = day*week_period # window length for the statistical tests

    X_train = [] # set the data of the initial window w0
    X_test = []

    for i in range(0, year-(7*96), day): 
        X_train.append(consumer_data[i:i+window_len])
        X_test.append(consumer_data[i+year:(i+year)+window_len])

    X_train = [l.tolist() for l in X_train]
    X_test = [l.tolist() for l in X_test]
    return X_train, X_test

def setupYData(consumer_data, consumer_id, fraud_point):
    day = 96 
    week_period = 7 # number of consecutive days to detect a change
    year = 96 * 365
    window_len = day*week_period # window length for the statistical tests

    c_train = []
    c_test = []
    for i in range(0, year-(7*96), day):  
        c_train.append(consumer_data[i:i+window_len])
        c_test.append(consumer_data[i+year:(i+year)+window_len])
    
    all_train_data = []
    for i in c_train:
        c_train_copy = ','.join(map(str, i))
        all_train_data.append(c_train_copy)
        
    c_train_df = {'ID': consumer_id,
                  'Consumption': all_train_data,
                   'Labels': -1}
    c_train_df = pd.DataFrame(c_train_df)
    c_train_df.to_csv('./'+ str(consumer_id) + '_train.csv')

    
    all_test_data = []
    for i in c_test:
        c_test_copy = ','.join(map(str, i))
        all_test_data.append(c_test_copy)
        
    c_test_df = {'ID': consumer_id,
                 'Consumption': all_test_data,
                   'Labels': -1}
    c_test_df = pd.DataFrame(c_test_df)
    fraud_point_df = int((fraud_point - (365*96))/(96*7))
#     print(fraud_point_df)
    c_test_df.loc[fraud_point_df:, 'Labels'] = 1
#     print(c_test_df)
    c_test_df.to_csv('./'+ str(consumer_id) + '_test.csv')

    y_train = c_train_df
    y_test = c_test_df
    return y_train, y_test

total_accuracy = []
total_precision = []
total_recall = []
total_f1 = []
tsf_pred = []

print('Time Series Forest\n\n')

count = 0
for i in all_data:
    count += 1
    print(f'Dataset: {count}')
    for j in range(len(i.index)):
        consumer_data, consumer_id, fraud_point = consumerData(i, j)
        fraud_point_week = (fraud_point/96)
        X_train, X_test = setupXData(consumer_data)
        y_train, y_test = setupYData(consumer_data, consumer_id, fraud_point)

        tsf = TimeSeriesForestClassifier()
        tsf.fit(X_train, y_train)
        pred = tsf.predict(X_test)


        # weekly_consumption_year1, weekly_consumption_year2 = setupWeeklyData(consumer_data)

        # features = []
        # for week_data in weekly_consumption_year1 + weekly_consumption_year2:
        #     result = catch22_all(week_data)
        #     features.append(result['values'])

        # features = np.array(features)
        # features = features[~np.isnan(features).any(axis=1)]
        # features = scaler.fit_transform(features)
        # features = pca.fit_transform(features)

        # tsf = TimeSeriesForestClassifier()
        # tsf.fit(features)
        # # Predict on the test data
        # pred = tsf.predict(features)

        # ground_truth = get_ground_truth(len(features), fraud_point_week)

        accuracy = accuracy_score(y_test, pred)
        precision = precision_score(y_test, pred)
        recall = recall_score(y_test, pred)
        f1 = f1_score(y_test, pred)

        print(f"Consumer: {consumer_id}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("--------------")

        # Assuming total_accuracy, total_precision, total_recall, and total_f1 are lists that have been defined prior to the loop
        total_accuracy.append(accuracy)
        total_precision.append(precision)
        total_recall.append(recall)
        total_f1.append(f1)

    print(f'Total accuracy: {sum(total_accuracy)/len(total_accuracy):.4f}')
    print(f'Total precision: {sum(total_precision)/len(total_precision):.4f}')
    print(f'Total recall: {sum(total_recall)/len(total_recall):.4f}')
    print(f'Total f1: {sum(total_f1)/len(total_f1):.4f}')

Time Series Forest




NameError: name 'all_data' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d7f56da1-e665-45c2-b1a9-a2c9d93d2a27' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>